In [1]:
##########################################################################################################
# VERSION  	DESARROLLADOR             FECHA        DESCRIPCION
# -------------------------------------------------------------
#  1        Walter Albites Azarte     10/12/2021   Curso PySpark Entorno Local - Dataframe
##########################################################################################################

In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.1.2-bin-hadoop2.7'

In [3]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf= pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark=SparkSession(sc)

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [5]:
### Crear Dataframe Spark personas
schemaPersona = StructType([
    StructField("dni", StringType(),True),
    StructField("nombre", StringType(),True),
    StructField("edad", IntegerType(),True)
])

dataPersona = ([("99999999", "Walter",35),
                ("88888888", "Susan",30),
                ("77777777", "Alejandro",12),
                ("66666666", "Pedro",30),
                ("55555555", "Karina",35),
                ("44444444", "Andy",26),
                ("33333333", " Raquel\n",45),
                ("22222222", "Gian",28),
                ("11111111", "Raul",45),
                ("00000000", "Elena",40),
                ("10101010", None, None),
                (None,'Raquel',None)])

df_personas=spark.createDataFrame(dataPersona,schema=schemaPersona)
df_personas.show()

+--------+---------+----+
|     dni|   nombre|edad|
+--------+---------+----+
|99999999|   Walter|  35|
|88888888|    Susan|  30|
|77777777|Alejandro|  12|
|66666666|    Pedro|  30|
|55555555|   Karina|  35|
|44444444|     Andy|  26|
|33333333|  Raquel
|  45|
|22222222|     Gian|  28|
|11111111|     Raul|  45|
|00000000|    Elena|  40|
|10101010|     null|null|
|    null|   Raquel|null|
+--------+---------+----+



In [6]:
df_personas.printSchema()

root
 |-- dni: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- edad: integer (nullable = true)



In [7]:
#Verificar si existe Saltos de Linea en las columnas
df_personas.select([count(when( col(c).contains('\n'), c)).alias(c) for c in df_personas.columns]).show(5,False)

+---+------+----+
|dni|nombre|edad|
+---+------+----+
|0  |1     |0   |
+---+------+----+



In [8]:
#Correcion si es que huniera una columna encontrada x ejemplo nombre
df_personas=df_personas.withColumn('nombre',regexp_replace('nombre', '\n', ''))
df_personas.show(20)

+--------+---------+----+
|     dni|   nombre|edad|
+--------+---------+----+
|99999999|   Walter|  35|
|88888888|    Susan|  30|
|77777777|Alejandro|  12|
|66666666|    Pedro|  30|
|55555555|   Karina|  35|
|44444444|     Andy|  26|
|33333333|   Raquel|  45|
|22222222|     Gian|  28|
|11111111|     Raul|  45|
|00000000|    Elena|  40|
|10101010|     null|null|
|    null|   Raquel|null|
+--------+---------+----+



In [9]:
# concat_ws() función de Pyspark concatena múltiples columnas de cadenas en una sola columna con un separador
# o delimitador determinado.
df_personas.withColumn('datos_personales', concat_ws(',',col('dni'),col('nombre'))).show(20,False)

+--------+---------+----+------------------+
|dni     |nombre   |edad|datos_personales  |
+--------+---------+----+------------------+
|99999999|Walter   |35  |99999999,Walter   |
|88888888|Susan    |30  |88888888,Susan    |
|77777777|Alejandro|12  |77777777,Alejandro|
|66666666|Pedro    |30  |66666666,Pedro    |
|55555555|Karina   |35  |55555555,Karina   |
|44444444|Andy     |26  |44444444,Andy     |
|33333333| Raquel  |45  |33333333, Raquel  |
|22222222|Gian     |28  |22222222,Gian     |
|11111111|Raul     |45  |11111111,Raul     |
|00000000|Elena    |40  |00000000,Elena    |
|10101010|null     |null|10101010          |
|null    |Raquel   |null|Raquel            |
+--------+---------+----+------------------+



In [10]:
df_personas.withColumn('datos_personales2', concat(col('dni'),col('nombre'))).show(20,False)

+--------+---------+----+-----------------+
|dni     |nombre   |edad|datos_personales2|
+--------+---------+----+-----------------+
|99999999|Walter   |35  |99999999Walter   |
|88888888|Susan    |30  |88888888Susan    |
|77777777|Alejandro|12  |77777777Alejandro|
|66666666|Pedro    |30  |66666666Pedro    |
|55555555|Karina   |35  |55555555Karina   |
|44444444|Andy     |26  |44444444Andy     |
|33333333| Raquel  |45  |33333333 Raquel  |
|22222222|Gian     |28  |22222222Gian     |
|11111111|Raul     |45  |11111111Raul     |
|00000000|Elena    |40  |00000000Elena    |
|10101010|null     |null|null             |
|null    |Raquel   |null|null             |
+--------+---------+----+-----------------+



In [11]:
# Coalesce función de Pyspark Devuelve la primera columna que no es nula.
df_documentoidentidad = spark.createDataFrame([(None, None), (88888888, None), (None, 99999999)], ("dni_1", "dni_2"))
df_documentoidentidad.show()

+--------+--------+
|   dni_1|   dni_2|
+--------+--------+
|    null|    null|
|88888888|    null|
|    null|99999999|
+--------+--------+



In [12]:
df_documentoidentidad.select(coalesce(col("dni_1"), col("dni_2")).alias("dni")).show()

+--------+
|     dni|
+--------+
|    null|
|88888888|
|99999999|
+--------+



In [13]:
df_personas.show()

+--------+---------+----+
|     dni|   nombre|edad|
+--------+---------+----+
|99999999|   Walter|  35|
|88888888|    Susan|  30|
|77777777|Alejandro|  12|
|66666666|    Pedro|  30|
|55555555|   Karina|  35|
|44444444|     Andy|  26|
|33333333|   Raquel|  45|
|22222222|     Gian|  28|
|11111111|     Raul|  45|
|00000000|    Elena|  40|
|10101010|     null|null|
|    null|   Raquel|null|
+--------+---------+----+



In [14]:
### Ejemplos utilizando UDF

In [15]:
#Crear una funcion que convierta en minusculas
def ConvertirMinusculas(texto):
    texto=texto.lower()
    return texto

In [16]:
# Crear un UDF a apartir de una funcion de Python
udf_convertir_misnusculas=udf(ConvertirMinusculas,StringType())

In [17]:
# Crear una columna normalizada en minuscula con la columna nombre y curso, Saldra error al parecer lee todas las filas
# del dataframe Los null y vacios por mas que se use la condicional

#df_personasy=df_personas.withColumn("nombre_normalizado",when(((col("nombre").isNotNull()) & (col("nombre")!="")),
#                                         udf_convertir_misnusculas("nombre")).otherwise(""))

#df_personasy.show()

# AttributeError: 'NoneType' object has no attribute 'lower'

In [18]:
#Solucion 1
#Crear una nueva columna temporal para colocar valores vacios a los NULL
df_personasz=df_personas.select('*',col("nombre").alias("nombre_temp"))
df_personasz=df_personasz.na.fill(value="",subset=["nombre_temp"])
df_personasz.show()

+--------+---------+----+-----------+
|     dni|   nombre|edad|nombre_temp|
+--------+---------+----+-----------+
|99999999|   Walter|  35|     Walter|
|88888888|    Susan|  30|      Susan|
|77777777|Alejandro|  12|  Alejandro|
|66666666|    Pedro|  30|      Pedro|
|55555555|   Karina|  35|     Karina|
|44444444|     Andy|  26|       Andy|
|33333333|   Raquel|  45|     Raquel|
|22222222|     Gian|  28|       Gian|
|11111111|     Raul|  45|       Raul|
|00000000|    Elena|  40|      Elena|
|10101010|     null|null|           |
|    null|   Raquel|null|     Raquel|
+--------+---------+----+-----------+



In [19]:
# Aplicamos el filtro y utilizamos el UDF 
df_personasz=df_personasz.withColumn("nombre_normalizado",
                                    when(col("nombre_temp")!="",
                                         udf_convertir_misnusculas("nombre_temp")).\
                                         #lit("Con Dato")).
                                    otherwise("")
                                   ).drop("nombre_temp")
df_personasz.show(20,False)

+--------+---------+----+------------------+
|dni     |nombre   |edad|nombre_normalizado|
+--------+---------+----+------------------+
|99999999|Walter   |35  |walter            |
|88888888|Susan    |30  |susan             |
|77777777|Alejandro|12  |alejandro         |
|66666666|Pedro    |30  |pedro             |
|55555555|Karina   |35  |karina            |
|44444444|Andy     |26  |andy              |
|33333333| Raquel  |45  | raquel           |
|22222222|Gian     |28  |gian              |
|11111111|Raul     |45  |raul              |
|00000000|Elena    |40  |elena             |
|10101010|null     |null|                  |
|null    |Raquel   |null|raquel            |
+--------+---------+----+------------------+



In [20]:
#Solucion 2
#Modificar la funcion cuando es None asignarle vacio
def ConvertirMinusculas(texto):
    if texto==None:
        texto=""
    else:
        texto=texto.lower()
    return texto

udf_convertir_misnusculas=udf(ConvertirMinusculas,StringType())

In [21]:
df_personasm=df_personas.withColumn("nombre_normalizado",
                                    when(((col("nombre").isNotNull()) & (col("nombre")!="")),
                                         udf_convertir_misnusculas("nombre")).\
                                    otherwise("")
                                   )
df_personasm.show(20,False)

+--------+---------+----+------------------+
|dni     |nombre   |edad|nombre_normalizado|
+--------+---------+----+------------------+
|99999999|Walter   |35  |walter            |
|88888888|Susan    |30  |susan             |
|77777777|Alejandro|12  |alejandro         |
|66666666|Pedro    |30  |pedro             |
|55555555|Karina   |35  |karina            |
|44444444|Andy     |26  |andy              |
|33333333| Raquel  |45  | raquel           |
|22222222|Gian     |28  |gian              |
|11111111|Raul     |45  |raul              |
|00000000|Elena    |40  |elena             |
|10101010|null     |null|                  |
|null    |Raquel   |null|raquel            |
+--------+---------+----+------------------+



In [22]:
df_personas_x=df_personas

In [23]:
# Renombrar Columnas utilizando withColumnRenamed, Consume mucha memoria
for column in df_personas_x.columns:
        df_personas_x=df_personas_x.withColumnRenamed(column,'personas_' + column)

In [24]:
df_personas_x.show(5,False)

+------------+---------------+-------------+
|personas_dni|personas_nombre|personas_edad|
+------------+---------------+-------------+
|99999999    |Walter         |35           |
|88888888    |Susan          |30           |
|77777777    |Alejandro      |12           |
|66666666    |Pedro          |30           |
|55555555    |Karina         |35           |
+------------+---------------+-------------+
only showing top 5 rows



In [25]:
# Renombrar columnas utilizando Alias, no consume mucha memoria Recomendable
df_personas_y=df_personas.select([col(i).alias('persons_'+i) for i in df_personas.columns])
df_personas_y.show()

+-----------+--------------+------------+
|persons_dni|persons_nombre|persons_edad|
+-----------+--------------+------------+
|   99999999|        Walter|          35|
|   88888888|         Susan|          30|
|   77777777|     Alejandro|          12|
|   66666666|         Pedro|          30|
|   55555555|        Karina|          35|
|   44444444|          Andy|          26|
|   33333333|        Raquel|          45|
|   22222222|          Gian|          28|
|   11111111|          Raul|          45|
|   00000000|         Elena|          40|
|   10101010|          null|        null|
|       null|        Raquel|        null|
+-----------+--------------+------------+



In [26]:
# Indentificar Columnas NULL, Nan y Vacios
df_personas.select([count(when( (isnan(c))| (col(c).isNull()) | (col(c)==''), c)).alias(c) for c in df_personas.columns]).show()

+---+------+----+
|dni|nombre|edad|
+---+------+----+
|  1|     1|   2|
+---+------+----+



In [27]:
#Verificar espacios al inicio y al final de una columna
df_personas.select([count(when( col(c).contains('\xa0'), c)).alias(c) for c in df_personas.columns]).show(5,False)

+---+------+----+
|dni|nombre|edad|
+---+------+----+
|0  |0     |0   |
+---+------+----+



In [28]:
# Adicionar una columna con la fecha del sistema
df_personas.withColumn('fecha',current_date()).show()

+--------+---------+----+----------+
|     dni|   nombre|edad|     fecha|
+--------+---------+----+----------+
|99999999|   Walter|  35|2022-07-20|
|88888888|    Susan|  30|2022-07-20|
|77777777|Alejandro|  12|2022-07-20|
|66666666|    Pedro|  30|2022-07-20|
|55555555|   Karina|  35|2022-07-20|
|44444444|     Andy|  26|2022-07-20|
|33333333|   Raquel|  45|2022-07-20|
|22222222|     Gian|  28|2022-07-20|
|11111111|     Raul|  45|2022-07-20|
|00000000|    Elena|  40|2022-07-20|
|10101010|     null|null|2022-07-20|
|    null|   Raquel|null|2022-07-20|
+--------+---------+----+----------+



In [29]:
# Adicionar una columna con la fecha y Hora  del sistema
df_personas.withColumn("fechaHora",to_timestamp(current_timestamp(),"MM-dd-yyyy HH mm ss SSS")).show(5,False)

+--------+---------+----+-----------------------+
|dni     |nombre   |edad|fechaHora              |
+--------+---------+----+-----------------------+
|99999999|Walter   |35  |2022-07-20 16:29:55.372|
|88888888|Susan    |30  |2022-07-20 16:29:55.372|
|77777777|Alejandro|12  |2022-07-20 16:29:55.372|
|66666666|Pedro    |30  |2022-07-20 16:29:55.372|
|55555555|Karina   |35  |2022-07-20 16:29:55.372|
+--------+---------+----+-----------------------+
only showing top 5 rows



In [30]:
# Capturar la fecha y hora de Inicio de ejecucion
from datetime import datetime
x=str(datetime.now())
print(x)

2022-07-20 16:29:56.171173


In [31]:
# fechas en Python
import time
print (time.strftime("%d/%m/%y"))

20/07/22


In [32]:
df_personas.withColumn('date',lit(time.strftime("%d/%m/%y"))).show()

+--------+---------+----+--------+
|     dni|   nombre|edad|    date|
+--------+---------+----+--------+
|99999999|   Walter|  35|20/07/22|
|88888888|    Susan|  30|20/07/22|
|77777777|Alejandro|  12|20/07/22|
|66666666|    Pedro|  30|20/07/22|
|55555555|   Karina|  35|20/07/22|
|44444444|     Andy|  26|20/07/22|
|33333333|   Raquel|  45|20/07/22|
|22222222|     Gian|  28|20/07/22|
|11111111|     Raul|  45|20/07/22|
|00000000|    Elena|  40|20/07/22|
|10101010|     null|null|20/07/22|
|    null|   Raquel|null|20/07/22|
+--------+---------+----+--------+



In [33]:
# Valores maximos de cada Columna
for i in df_personas.columns:
    df_personas.select(max(length(i))).show()

+----------------+
|max(length(dni))|
+----------------+
|               8|
+----------------+

+-------------------+
|max(length(nombre))|
+-------------------+
|                  9|
+-------------------+

+-----------------+
|max(length(edad))|
+-----------------+
|                2|
+-----------------+



In [34]:
################### Join parametrizable en Funciones Pyspark

In [35]:
#Crear Dataframe Spark Cursos
schemaCursos = StructType([
    StructField("dni", StringType(),True),
    StructField("curso", StringType(),True),
    StructField("precio", DoubleType(),True)
])

dataCurso = ([("99999999", "Spark",100.50),
              ("99999999", "Scala",100.50),
              ("99999999", "Java",100.50),
              ("88888888", "Ingles",80.90),
              ("77777777", "Java",12.50)
])

df_cursos=spark.createDataFrame(dataCurso,schema=schemaCursos)
df_cursos.show()

+--------+------+------+
|     dni| curso|precio|
+--------+------+------+
|99999999| Spark| 100.5|
|99999999| Scala| 100.5|
|99999999|  Java| 100.5|
|88888888|Ingles|  80.9|
|77777777|  Java|  12.5|
+--------+------+------+



In [36]:
def join_example(dfa,dfb):
    dfa_whith_dfb=dfa.alias("a").join(dfb.alias("b"),col("a.dni")==col("b.dni"),how='inner')
    return dfa_whith_dfb

dfa_whith_dfb=join_example(df_personas,df_cursos)
dfa_whith_dfb.show()

+--------+---------+----+--------+------+------+
|     dni|   nombre|edad|     dni| curso|precio|
+--------+---------+----+--------+------+------+
|88888888|    Susan|  30|88888888|Ingles|  80.9|
|99999999|   Walter|  35|99999999| Spark| 100.5|
|99999999|   Walter|  35|99999999| Scala| 100.5|
|99999999|   Walter|  35|99999999|  Java| 100.5|
|77777777|Alejandro|  12|77777777|  Java|  12.5|
+--------+---------+----+--------+------+------+



In [37]:
################### Join parametrizable en Funciones Pandas to Pyspark

In [38]:
df_personas_pandas=df_personas.toPandas()
df_cursos_pandas=df_cursos.toPandas()

def merge_example_pandas(dfa,dfb):
    dfa_whith_dfb=dfa.merge(dfb,how='inner',on="dni")
    return dfa_whith_dfb

dfa_whith_dfb_pandas=merge_example_pandas(df_personas_pandas,df_cursos_pandas)

dfa_whith_dfb_pandas.head()

,dni,nombre,edad,curso,precio
0,99999999,Walter,35.0,Spark,100.5
1,99999999,Walter,35.0,Scala,100.5
2,99999999,Walter,35.0,Java,100.5
3,88888888,Susan,30.0,Ingles,80.9
4,77777777,Alejandro,12.0,Java,12.5


In [39]:
#Convertir a dataframe de Spark
dfa_whith_dfb_pandas_spark = spark.createDataFrame(dfa_whith_dfb_pandas)
dfa_whith_dfb_pandas_spark.show()

+--------+---------+----+------+------+
|     dni|   nombre|edad| curso|precio|
+--------+---------+----+------+------+
|99999999|   Walter|35.0| Spark| 100.5|
|99999999|   Walter|35.0| Scala| 100.5|
|99999999|   Walter|35.0|  Java| 100.5|
|88888888|    Susan|30.0|Ingles|  80.9|
|77777777|Alejandro|12.0|  Java|  12.5|
+--------+---------+----+------+------+



In [40]:
#Conteos
total=df_personas.count()

for i in df_personas.columns:
    conteo=df_personas.select(i).filter(col(i).isNotNull()).count()
    print(i,conteo,(conteo/total)*100)

dni 11 91.66666666666666
nombre 11 91.66666666666666
edad 10 83.33333333333334


In [41]:
# leer CSV para que reconozca los satos de linea
#df_personas = spark.read.options(header='True',inferSchema='True',delimiter=',',escape='"',quote='"',multiLine=True).\
#csv(root + 'personas.csv')

In [42]:
# Inicio Tranformar DataFrame de Pandas a Dataframe de Spark
# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types):
        struct_list.append(define_structure(column, typo))
        
    p_schema = StructType(struct_list)
    return spark.createDataFrame(pandas_df, p_schema)

In [43]:
#Creamos un DataFrame en Pandas
import pandas as pd

df1 = pd.DataFrame({
   'Jugadores': ["Federer", "Ronaldo", "Phelps", "Valenzuela"],
   'Posicion': ["Delantero", "Delantero", "Medio campista", "Defensa"]
})
print(df1.dtypes)
df1.head()

Jugadores    object
Posicion     object
dtype: object


,Jugadores,Posicion
0,Federer,Delantero
1,Ronaldo,Delantero
2,Phelps,Medio campista
3,Valenzuela,Defensa


In [44]:
DF1_SPARK=pandas_to_spark(df1)
DF1_SPARK.show()

+----------+--------------+
| Jugadores|      Posicion|
+----------+--------------+
|   Federer|     Delantero|
|   Ronaldo|     Delantero|
|    Phelps|Medio campista|
|Valenzuela|       Defensa|
+----------+--------------+



In [45]:
DF1_SPARK.printSchema()
# Fin Tranformar DataFrame de Pandas a Dataframe de Spark

root
 |-- Jugadores: string (nullable = true)
 |-- Posicion: string (nullable = true)



In [46]:
#Crear Dataframe Spark Solicitudes
schemaSolicitudes = StructType([
    StructField("cif", StringType(),True),
    StructField("entidad_solicitud", StringType(),True),
    StructField("origen_solicitud", StringType(),True)
])

dataSolicitudes = ([("A79122305", "UNIVERSIDAD EUROPEA DE MADRID SA","ACT_NAC"),
                ("Q2868013J", "ACADEMIA DE BELLAS ARTES DE SAN FERNANDO","ACT_NAC"),
                ("G46129698", "FUNDACIÓN INSTITUTO VALENCIANO DE ONCOLOGÍA","ACT_NAC")
               ])

df_solicitudes=spark.createDataFrame(dataSolicitudes,schema=schemaSolicitudes)
df_solicitudes.show()

#Crear Dataframe Spark Entidades
schemaEntidades = StructType([
    StructField("cif", StringType(),True),
    StructField("acronimo_entidad", StringType(),True),
    StructField("nombre_entidad", StringType(),True)
])

dataEntidades = ([("A79122305", "MATERPLAT","CENTRO NACIONAL DE EPIDEMIOLOGIA"),
              ("A79122305", "MATERPLAT","CENTRO NACIONAL DE EPIDEMIOLOGIA"),
              ("A79122305", "MATERPLAT","CENTRO NACIONAL DE EPIDEMIOLOGIA"),
              ("Q2868013J", "CNE","RED DE LABORATORIOS DE ALERTAS BIOLOGICAS"),
              ("Q2868013J", "CNE","RED DE LABORATORIOS DE ALERTAS BIOLOGICAS")
])

df_entidades=spark.createDataFrame(dataEntidades,schema=schemaEntidades)
df_entidades.show()

+---------+--------------------+----------------+
|      cif|   entidad_solicitud|origen_solicitud|
+---------+--------------------+----------------+
|A79122305|UNIVERSIDAD EUROP...|         ACT_NAC|
|Q2868013J|ACADEMIA DE BELLA...|         ACT_NAC|
|G46129698|FUNDACIÓN INSTITU...|         ACT_NAC|
+---------+--------------------+----------------+

+---------+----------------+--------------------+
|      cif|acronimo_entidad|      nombre_entidad|
+---------+----------------+--------------------+
|A79122305|       MATERPLAT|CENTRO NACIONAL D...|
|A79122305|       MATERPLAT|CENTRO NACIONAL D...|
|A79122305|       MATERPLAT|CENTRO NACIONAL D...|
|Q2868013J|             CNE|RED DE LABORATORI...|
|Q2868013J|             CNE|RED DE LABORATORI...|
+---------+----------------+--------------------+



In [47]:
DF_A_B_NO_CRUZAN=df_solicitudes.join(df_entidades,on="cif",how="anti")
DF_A_B_NO_CRUZAN.show()

+---------+--------------------+----------------+
|      cif|   entidad_solicitud|origen_solicitud|
+---------+--------------------+----------------+
|G46129698|FUNDACIÓN INSTITU...|         ACT_NAC|
+---------+--------------------+----------------+



In [48]:
DF_A_B_CRUZAN=df_solicitudes.alias('a').join(df_entidades.alias('b'),
                                             col('a.cif')==col('b.cif'),
                                             how="inner"
                                            ).select('a.*')
DF_A_B_CRUZAN.show()

+---------+--------------------+----------------+
|      cif|   entidad_solicitud|origen_solicitud|
+---------+--------------------+----------------+
|Q2868013J|ACADEMIA DE BELLA...|         ACT_NAC|
|Q2868013J|ACADEMIA DE BELLA...|         ACT_NAC|
|A79122305|UNIVERSIDAD EUROP...|         ACT_NAC|
|A79122305|UNIVERSIDAD EUROP...|         ACT_NAC|
|A79122305|UNIVERSIDAD EUROP...|         ACT_NAC|
+---------+--------------------+----------------+



In [49]:
###################################################### Las funciones de PySpark Window 
#operan en un grupo de filas (como marco, división) y 
# devuelven un valor único para cada fila de entrada.
simpleData = (("James", "Ventas", 3000), \
    ("Michael", "Ventas", 4600),  \
    ("Robert", "Ventas", 4100),   \
    ("Maria", "Sistemas", 3000),  \
    ("James", "Ventas", 3000),    \
    ("Scott", "Sistemas", 3300),  \
    ("Jen", "Sistemas", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Ventas", 4100) \
  )
 
columns= ["nombre_empleado", "departamento", "salario"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- nombre_empleado: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- salario: long (nullable = true)

+---------------+------------+-------+
|nombre_empleado|departamento|salario|
+---------------+------------+-------+
|James          |Ventas      |3000   |
|Michael        |Ventas      |4600   |
|Robert         |Ventas      |4100   |
|Maria          |Sistemas    |3000   |
|James          |Ventas      |3000   |
|Scott          |Sistemas    |3300   |
|Jen            |Sistemas    |3900   |
|Jeff           |Marketing   |3000   |
|Kumar          |Marketing   |2000   |
|Saif           |Ventas      |4100   |
+---------------+------------+-------+



In [50]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
#Para realizar una operación en un grupo primero, necesitamos particionar los datos usando Window.partitionBy(),
#y para el número de fila y la función de rango, necesitamos ordenar adicionalmente por la orderBycláusula 
#de uso de datos de partición.
windowPartitionDepartamento  = Window.partitionBy("departamento").orderBy("salario")
#row_number()La función de window se utiliza para dar el número de fila secuencial a
#partir de 1 al resultado de cada división de wimdow.
df.withColumn("row_number",row_number().over(windowPartitionDepartamento)).orderBy('departamento').show(truncate=False)

+---------------+------------+-------+----------+
|nombre_empleado|departamento|salario|row_number|
+---------------+------------+-------+----------+
|Kumar          |Marketing   |2000   |1         |
|Jeff           |Marketing   |3000   |2         |
|Maria          |Sistemas    |3000   |1         |
|Scott          |Sistemas    |3300   |2         |
|Jen            |Sistemas    |3900   |3         |
|James          |Ventas      |3000   |1         |
|James          |Ventas      |3000   |2         |
|Robert         |Ventas      |4100   |3         |
|Saif           |Ventas      |4100   |4         |
|Michael        |Ventas      |4600   |5         |
+---------------+------------+-------+----------+



In [51]:
df.show()

+---------------+------------+-------+
|nombre_empleado|departamento|salario|
+---------------+------------+-------+
|          James|      Ventas|   3000|
|        Michael|      Ventas|   4600|
|         Robert|      Ventas|   4100|
|          Maria|    Sistemas|   3000|
|          James|      Ventas|   3000|
|          Scott|    Sistemas|   3300|
|            Jen|    Sistemas|   3900|
|           Jeff|   Marketing|   3000|
|          Kumar|   Marketing|   2000|
|           Saif|      Ventas|   4100|
+---------------+------------+-------+



In [52]:
#Salario por departamento
windowPartitionDepartamentoX  = Window.partitionBy("departamento")
df.withColumn("salario_maximo",max('salario').over(windowPartitionDepartamentoX)).orderBy('departamento').show(truncate=False)

+---------------+------------+-------+--------------+
|nombre_empleado|departamento|salario|salario_maximo|
+---------------+------------+-------+--------------+
|Jeff           |Marketing   |3000   |3000          |
|Kumar          |Marketing   |2000   |3000          |
|Maria          |Sistemas    |3000   |3900          |
|Scott          |Sistemas    |3300   |3900          |
|Jen            |Sistemas    |3900   |3900          |
|James          |Ventas      |3000   |4600          |
|Michael        |Ventas      |4600   |4600          |
|Robert         |Ventas      |4100   |4600          |
|James          |Ventas      |3000   |4600          |
|Saif           |Ventas      |4100   |4600          |
+---------------+------------+-------+--------------+



In [53]:
######### collect_set, collect_list
df.select('departamento','nombre_empleado','salario').\
groupBy('departamento').\
agg(collect_set('nombre_empleado').alias('lista_empleados'),
    collect_list('salario').alias('lista_salario'),
    collect_set('salario').alias('lista_salario_unicos')
   ).show(5,False)

+------------+------------------------------+------------------------------+--------------------+
|departamento|lista_empleados               |lista_salario                 |lista_salario_unicos|
+------------+------------------------------+------------------------------+--------------------+
|Ventas      |[Robert, Michael, Saif, James]|[3000, 4600, 4100, 3000, 4100]|[4600, 3000, 4100]  |
|Marketing   |[Kumar, Jeff]                 |[3000, 2000]                  |[3000, 2000]        |
|Sistemas    |[Maria, Scott, Jen]           |[3000, 3300, 3900]            |[3900, 3000, 3300]  |
+------------+------------------------------+------------------------------+--------------------+

